In [1]:
import os
import numpy as np
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Initialize the pre-trained MobileNet model without the top classification layer
model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3), pooling='avg')

# Function to extract features from an image
def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)  # Preprocess for MobileNet
    features = model.predict(img_array)
    return features.flatten()

# Directory paths
hamburger_train_dir = '../data/food11/train/hamburger'
hamburger_test_dir = '../data/food11/test/hamburger'

# Collect all image paths in the pizza folders
image_paths = [os.path.join(hamburger_train_dir, fname) for fname in os.listdir(hamburger_train_dir)]
image_paths += [os.path.join(hamburger_test_dir, fname) for fname in os.listdir(hamburger_test_dir)]

# Extract features for all images
features = []
for img_path in image_paths:
    try:
        features.append(extract_features(img_path, model))
    except Exception as e:
        print(f"Error processing image {img_path}: {e}")

features = np.array(features)
print(f"Extracted features for {len(features)} images")

Extracted features for 911 images


In [2]:
from sklearn.cluster import KMeans

# Number of clusters (choose based on expected subcategories, e.g., 3 types of pizza)
num_clusters = 8

# Perform K-Means clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(features)

# Get cluster labels for each image
cluster_labels = kmeans.labels_

# Print the cluster label for each image
for img_path, label in zip(image_paths, cluster_labels):
    print(f"{img_path} assigned to cluster {label}")

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


../data/food11/train/hamburger\1007277.jpg assigned to cluster 3
../data/food11/train/hamburger\1009426.jpg assigned to cluster 4
../data/food11/train/hamburger\1010136.jpg assigned to cluster 3
../data/food11/train/hamburger\1010140.jpg assigned to cluster 3
../data/food11/train/hamburger\1010695.jpg assigned to cluster 0
../data/food11/train/hamburger\1013830.jpg assigned to cluster 0
../data/food11/train/hamburger\1017874.jpg assigned to cluster 4
../data/food11/train/hamburger\1028639.jpg assigned to cluster 4
../data/food11/train/hamburger\1029228.jpg assigned to cluster 5
../data/food11/train/hamburger\1031616.jpg assigned to cluster 2
../data/food11/train/hamburger\1035486.jpg assigned to cluster 0
../data/food11/train/hamburger\1036215.jpg assigned to cluster 4
../data/food11/train/hamburger\1036785.jpg assigned to cluster 0
../data/food11/train/hamburger\1038553.jpg assigned to cluster 2
../data/food11/train/hamburger\1041547.jpg assigned to cluster 4
../data/food11/train/hamb

In [3]:
import shutil

# Create directories for each cluster (subcategories)
for i in range(num_clusters):
    train_subdir = os.path.join(hamburger_train_dir, f"cluster_{i}")
    test_subdir = os.path.join(hamburger_test_dir, f"cluster_{i}")
    os.makedirs(train_subdir, exist_ok=True)
    os.makedirs(test_subdir, exist_ok=True)

# Move images to subdirectories based on cluster labels
for img_path, label in zip(image_paths, cluster_labels):
    if 'train' in img_path:
        shutil.move(img_path, os.path.join(hamburger_train_dir, f"cluster_{label}", os.path.basename(img_path)))
    elif 'test' in img_path:
        shutil.move(img_path, os.path.join(hamburger_test_dir, f"cluster_{label}", os.path.basename(img_path)))

print("Images moved to their respective cluster directories.")

Images moved to their respective cluster directories.


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

# Fine-grained apple_pie classification
hamburger_train_gen = ImageDataGenerator(rescale=1./255)
hamburger_test_gen = ImageDataGenerator(rescale=1./255)

hamburger_train_generator = hamburger_train_gen.flow_from_directory(
    hamburger_train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

hamburger_test_generator = hamburger_test_gen.flow_from_directory(
    hamburger_test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Use the same base model (MobileNet)
base_model_hamburger = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model_hamburger.layers:
    layer.trainable = False

# Add custom layers for apple_pie classification
x_hamburger = base_model_hamburger.output
x_hamburger = GlobalAveragePooling2D()(x_hamburger)
x_hamburger = Dense(512, activation='relu')(x_hamburger)
hamburger_predictions = Dense(num_clusters, activation='softmax')(x_hamburger)

# Create and compile the model
hamburger_model = Model(inputs=base_model_hamburger.input, outputs=hamburger_predictions)
hamburger_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the fine-grained apple_pie classification model
hamburger_model.fit(
    hamburger_train_generator,
    steps_per_epoch=hamburger_train_generator.samples // 32,
    validation_data=hamburger_test_generator,
    validation_steps=hamburger_test_generator.samples // 32,
    epochs=30
)

# Save the model
hamburger_model.save('fine_grained_hamburger_classifier.h5')

Found 833 images belonging to 8 classes.
Found 78 images belonging to 8 classes.
Epoch 1/30
26/26 [==============================] - 8s 262ms/step - loss: 1.9445 - accuracy: 0.2984 - val_loss: 1.4565 - val_accuracy: 0.4844
Epoch 2/30
26/26 [==============================] - 7s 252ms/step - loss: 1.2425 - accuracy: 0.6117 - val_loss: 1.1177 - val_accuracy: 0.6250
Epoch 3/30
26/26 [==============================] - 6s 247ms/step - loss: 0.9251 - accuracy: 0.7366 - val_loss: 0.8686 - val_accuracy: 0.7344
Epoch 4/30
26/26 [==============================] - 7s 249ms/step - loss: 0.7041 - accuracy: 0.8190 - val_loss: 0.7350 - val_accuracy: 0.7812
Epoch 5/30
26/26 [==============================] - 6s 248ms/step - loss: 0.5636 - accuracy: 0.8789 - val_loss: 0.6203 - val_accuracy: 0.8594
Epoch 6/30
26/26 [==============================] - 6s 247ms/step - loss: 0.4592 - accuracy: 0.9251 - val_loss: 0.6175 - val_accuracy: 0.8438
Epoch 7/30
26/26 [==============================] - 7s 249ms/step -